## Project 2 Testing Subway
Jacob Minkin
Last project I tried to predict a critical violation from location and cuisine. The problem is that almost all restaurants have at least one critical violation which made it hard to get any predictive value from the model. Now I want to try again with predicting the score and therefore grade of the restaurant

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from sklearn.impute import KNNImputer
sns.set_style('darkgrid')

#Import Data
df = pd.read_csv('../data/Inspection.csv')
sub_df = pd.read_csv('../data/Subway_Distances.csv')
df['date'] = pd.to_datetime(df['INSPECTION DATE'])
df = df[df.BORO == 'Manhattan']

## Find most recent score for every restaurant

In [93]:
df.sort_values(['date'], ascending=False, inplace= True)
df = df.groupby(['CAMIS'], sort=False).first()
df = df.loc[(df.Longitude != 0) & (df.Latitude != 0)]

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10612 entries, 50114345 to 50107551
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   DBA                     10062 non-null  object        
 1   BORO                    10612 non-null  object        
 2   BUILDING                10597 non-null  object        
 3   STREET                  10612 non-null  object        
 4   ZIPCODE                 10612 non-null  float64       
 5   PHONE                   10610 non-null  object        
 6   CUISINE                 8911 non-null   object        
 7   INSPECTION DATE         10612 non-null  object        
 8   ACTION                  8911 non-null   object        
 9   VIOLATION CODE          8893 non-null   object        
 10  VIOLATION               8893 non-null   object        
 11  CRITICAL FLAG           10612 non-null  object        
 12  SCORE                   8910 non-nul

In [95]:
feature_list = ['SCORE', 'Latitude', 'Longitude']
df = df[feature_list].dropna()

In [96]:
import geopy.distance

def get_distance_and_index(longitude, latitude):
    coords_1 = (latitude, longitude)
    distances = []
    for index, row in sub_df.iterrows():
        coords_2 = (row['latitude'], row['longitude'])
        distance = geopy.distance.geodesic(coords_1, coords_2).km
        distances.append(distance)
    min_distance = min(distances)
    min_index = distances.index(min_distance)
    lines = sub_df.LINE.iloc[min_index]
    return min_distance, lines


In [97]:
get_distance_and_index( -74.0060, 40.7128)

(1.1906227203150102, "['B', 'D']")

In [98]:
def get_distance_and_index(df_row):
    latitude = df_row[1]
    longitude = df_row[0]
    coords_1 = (latitude, longitude)
    distances = []
    for index, row in sub_df.iterrows():
        coords_2 = (row['latitude'], row['longitude'])
        distance = geopy.distance.geodesic(coords_1, coords_2).km
        distances.append(distance)
    min_distance = min(distances)
    min_index = distances.index(min_distance)
    lines = sub_df.LINE.iloc[min_index]
    return min_distance, lines


In [99]:
df[['Longitude', 'Latitude']]

,Longitude,Latitude
CAMIS,,
50114345,-73.982292,40.742881
50067994,-73.929582,40.855003
50036864,-73.939258,40.816310
50057298,-73.959328,40.773421
50085148,-73.960339,40.761808
...,...,...
41226031,-73.990523,40.767425
40560953,-73.987446,40.758757
40944945,-73.993607,40.758823


In [100]:
list_mins = df[['Longitude', 'Latitude']].apply(get_distance_and_index, axis=1)

fancier restaurants may be closer to a major station
rats might live by entrances
more populated restaurants

In [101]:
list_mins

CAMIS
50114345                          (0.1291121840372822, ['6'])
50067994                        (0.041430558356215084, ['1'])
50036864                    (0.26163783553495323, ['2', '3'])
50057298                        (0.029038211504573412, ['6'])
50085148    (0.47368384437375183, ['N', 'Q', 'R', '4', '5'...
                                  ...                        
41226031      (0.6054896675759363, ['A', 'B', 'C', 'D', '1'])
40560953    (0.11092329311881607, ['A', 'C', 'E', 'N', 'Q'...
40944945    (0.341220674572975, ['A', 'C', 'E', 'N', 'Q', ...
50046091                         (0.22304983597024786, ['A'])
50043445                    (0.22708705354939313, ['A', 'C'])
Length: 8893, dtype: object

In [103]:
list_mins.to_csv('../data/Restaurant_Distances.csv', index=True)

In [43]:
df[['Longitude', 'Latitude']]

,Longitude,Latitude
CAMIS,,
50114345,-73.982292,40.742881
50067994,-73.929582,40.855003
50036864,-73.939258,40.816310
50057298,-73.959328,40.773421
50085148,-73.960339,40.761808
...,...,...
50130678,-73.988435,40.733804
50124010,-74.008325,40.709632
50129856,-73.977645,40.758709


In [202]:
char = list_mins[40560953][1]#.split("[")[0]
#.str.split('(', expand=True)[1].str.split(" ")
print(char)

#.strip("[]'")
#char
# list_df = pd.DataFrame(list_mins)
# list_df

['A', 'C', 'E', 'N', 'Q', 'R', 'S', '1', '2', '3', '7']


In [216]:
CAMIS_list = list_mins.index
CAMIS_distance = {}
CAMIS_lines = {}
for i, CAMIS in enumerate(CAMIS_list):
    CAMIS_distance[CAMIS] = list_mins[CAMIS][0]
    CAMIS_lines[CAMIS] = list_mins[CAMIS][1]
CAMIS_distance
CAMIS_lines.items()

dict_items([(50114345, "['6']"), (50067994, "['1']"), (50036864, "['2', '3']"), (50057298, "['6']"), (50085148, "['N', 'Q', 'R', '4', '5', '6']"), (41426958, "['A', 'C', 'E', 'N', 'Q', 'R', 'S', '1', '2', '3', '7']"), (50079290, "['6']"), (50081432, "['B', 'D']"), (50006741, "['A', 'C', 'E', 'N', 'Q', 'R', 'S', '1', '2', '3', '7']"), (41125878, "['L']"), (50057443, "['N', 'Q', 'R', '4', '5', '6']"), (50116824, "['N', 'R']"), (41166881, "['C']"), (50018738, "['A', 'C', 'E', 'N', 'Q', 'R', 'S', '1', '2', '3', '7']"), (50127582, "['1']"), (40371454, "['A', 'C', 'E', 'N', 'Q', 'R', 'S', '1', '2', '3', '7']"), (50078597, "['Q']"), (50055426, "['A', 'B', 'C', 'D', '1']"), (50127947, "['N', 'R']"), (50084330, "['1']"), (50097018, "['B', 'D', 'F', 'M']"), (50107564, "['1']"), (50084908, "['B', 'D']"), (50111322, "['1']"), (50046273, "['A', 'B', 'C', 'D', '1']"), (50104384, "['N', 'Q', 'R', '4', '5', '6']"), (50083186, "['Q']"), (50131439, "['1']"), (50072616, "['1']"), (50108947, "['6']"), (50

In [236]:

keys = ['M', 'R', 'F', '6', 'B', 'C', '1', '2', 'D', 'E', '5', 'A', '7', 'N', 'Q', '4', '3', 'L', 'G', 'J', 'Z', 'S']
sub_list = {key: [] for key in keys}

for key, value in CAMIS_lines.items():
    #print('KEY:', {key}, 'VALUE:', {value})
    for char in value:
        char = char.strip("[]' ")
        if char.isalpha() or char.isdigit():
            print('KEY:', {key}, 'CHAR:', {char}) 
            sub_list[char].append(key)

print(sub_list)

KEY: {50114345} CHAR: {'6'}
KEY: {50067994} CHAR: {'1'}
KEY: {50036864} CHAR: {'2'}
KEY: {50036864} CHAR: {'3'}
KEY: {50057298} CHAR: {'6'}
KEY: {50085148} CHAR: {'N'}
KEY: {50085148} CHAR: {'Q'}
KEY: {50085148} CHAR: {'R'}
KEY: {50085148} CHAR: {'4'}
KEY: {50085148} CHAR: {'5'}
KEY: {50085148} CHAR: {'6'}
KEY: {41426958} CHAR: {'A'}
KEY: {41426958} CHAR: {'C'}
KEY: {41426958} CHAR: {'E'}
KEY: {41426958} CHAR: {'N'}
KEY: {41426958} CHAR: {'Q'}
KEY: {41426958} CHAR: {'R'}
KEY: {41426958} CHAR: {'S'}
KEY: {41426958} CHAR: {'1'}
KEY: {41426958} CHAR: {'2'}
KEY: {41426958} CHAR: {'3'}
KEY: {41426958} CHAR: {'7'}
KEY: {50079290} CHAR: {'6'}
KEY: {50081432} CHAR: {'B'}
KEY: {50081432} CHAR: {'D'}
KEY: {50006741} CHAR: {'A'}
KEY: {50006741} CHAR: {'C'}
KEY: {50006741} CHAR: {'E'}
KEY: {50006741} CHAR: {'N'}
KEY: {50006741} CHAR: {'Q'}
KEY: {50006741} CHAR: {'R'}
KEY: {50006741} CHAR: {'S'}
KEY: {50006741} CHAR: {'1'}
KEY: {50006741} CHAR: {'2'}
KEY: {50006741} CHAR: {'3'}
KEY: {50006741} CHAR

In [243]:
# Save dictionaries
import pickle
CAMIS_distance
with open('distance.pickle', 'wb') as handle:
    pickle.dump(CAMIS_distance, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('subway_lines.pickle', 'wb') as handle:
    pickle.dump(sub_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
